# Stable Diffusion web UI on Colab

- tutorial: https://stable-diffusion-art.com/automatic1111-colab/
- purchase: https://andrewongai.gumroad.com/l/stable_diffusion_quick_start
- notebook: https://colab.research.google.com/github/sagiodev/stablediffusion_webui/blob/master/StableDiffusionUI_ngrok_sagiodev.ipynb

In [ ]:
import os, subprocess, time, glob
from google.colab import drive
from IPython.display import clear_output

# -----------------------------------------------------------------------------
# configs

output_path = 'SDFiles'

webui_version = 'v1.6.0'

models_from_url = 'https://civitai.com/api/download/models/72396' # separated by comma
save_a_copy_in_google_drive = False

extensions_from_url = '' # separated by comma

# -----------------------------------------------------------------------------
# functions

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
    if '3.10' in file.read():
      return

  # install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  # change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def linkGoogleDrive():
  # config files
  if not os.path.exists(output_path + '/ui-config.json'):
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {output_path + '/ui-config.json'}
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  if not os.path.exists(output_path + '/config.json'):
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {output_path + '/config.json'}
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  if not os.path.exists(output_path + '/styles.csv'):
    !wget https://raw.githubusercontent.com/sagiodev/stablediffusion_webui/master/styles.csv -O {output_path + '/styles.csv'}
  !ln -s {output_path + '/styles.csv'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs

  # embeddings
  !mkdir -p {output_path}/embeddings
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # parameter file
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # models
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  for f in models_in_google_drive:
    !ln -s {f}

  # upscalers
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # LoRAs
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # hypernetworks
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # VAEs
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

# -----------------------------------------------------------------------------
# main

output_path = '/content/drive/MyDrive/' + output_path
drive.mount('/content/drive')
root = '/content/'

!mkdir -p {output_path}/outputs
!mkdir -p {output_path}/models
!mkdir -p {output_path}/ESRGAN
!mkdir -p {output_path}/hypernetworks

updatePython()

!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2

# install webui
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd {root}/stable-diffusion-webui
!git checkout -f {webui_version}
!COMMANDLINE_ARGS="--exit"  python launch.py

linkGoogleDrive()

# download models
%cd {root}/stable-diffusion-webui/models/Stable-diffusion
downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
downloadModel('https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors')
downloadModel('https://civitai.com/api/download/models/128713?type=Model&format=SafeTensor&size=pruned&fp=fp16')
downloadModel('https://civitai.com/api/download/models/90854?type=Model&format=SafeTensor&size=full&fp=fp16')

if models_from_url:
  for url in models_from_url.split(','):
    %cd {root}/stable-diffusion-webui/models/Stable-diffusion
    downloadModel(url)
    if save_a_copy_in_google_drive:
      %cd {output_path}/models
      downloadModel(url)

# download VAEs
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

# install controlnet
%cd {root}/stable-diffusion-webui/extensions
!git clone https://github.com/Mikubill/sd-webui-controlnet.git
%cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
!pip install -r requirements.txt

# download controlnet models
%cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
# downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
# downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')
# downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
# downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')
# downloadModel('https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_qrpattern.safetensors')
downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth')
downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15_plus.pth')

# install other extensions
%cd {root}/stable-diffusion-webui/extensions
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111
%cd {root}/stable-diffusion-webui/extensions
!git clone https://github.com/fkunn1326/openpose-editor
%cd {root}/stable-diffusion-webui/extensions
!git clone https://github.com/Bing-su/adetailer

%cd {root}/stable-diffusion-webui/extensions
for url in extensions_from_url.split(','):
  !git clone {url}

# downgrade httpx to avoid TypeError: AsyncConnectionPool.__init__() got an unexpected keyword argument 'socket_options'
!pip install httpx==0.24.1

# clear output
clear_output()

# low ram patch
!sed -i 's/dict()))$/dict())).cuda()/g'  {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py

%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --share --disable-model-loading-ram-optimization --opt-sdp-no-mem-attention'
!python {root}/stable-diffusion-webui/launch.py {args}